In [94]:
from scrapfly import ScrapflyClient, ScrapeConfig, ScrapflyError
 
scrapfly = ScrapflyClient(key="scp-live-01fc8534987042f4a2fe553c5cf6a2df", max_concurrency=2)
url = 'https://www.biglots.com/product/p810689503'
listing_result = scrapfly.scrape(ScrapeConfig(url=url, render_js=True, country="US", asp=True, retry=False, rendering_wait=10000))
listing_html = listing_result.scrape_result['content']

In [95]:
from bs4 import BeautifulSoup
import re
import json
def extract_json_from_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    script_tags = soup.find_all('script')

    json_data_list = []

    for script in script_tags:
        script_content = script.string
        if script_content:
            # Use a regular expression to find JSON-like content
            json_pattern = re.compile(r'({.*})', re.DOTALL)
            matches = json_pattern.findall(script_content)
            for match in matches:
                try:
                    json_data = json.loads(match)
                    json_data_list.append(json_data)
                except json.JSONDecodeError:
                    pass  # Ignore invalid JSON

    return json_data_list

json_data_list = extract_json_from_html(listing_html)

In [99]:
sellername = None
site_url = None

for index in range(2):  # Assuming only the first two indices are relevant
    # Ensure the index is valid and that the 'name' is not None
    if index < len(json_data_list) and json_data_list[index].get('skuId') is not None:
        # Check if the current product name matches the desired one
            data = json_data_list[index]
            
            # Attempt to populate sellername and site_url if they exist
            if index + 1 < len(json_data_list):
                sellername = json_data_list[index + 1].get('@graph', [{}])[0].get('name')
                site_url = json_data_list[index + 1].get('@graph', [{}])[0].get('url')

            break


In [100]:
def flatten_json(nested_json, parent_key='', sep='_'):
    flattened_dict = {}

    # Iterate through all key-value pairs in the JSON object
    if isinstance(nested_json, dict):
        for key, value in nested_json.items():
            new_key = f"{parent_key}{sep}{key}" if parent_key else key  # Build new key with the parent key

            # If value is a dict, recursively flatten it
            if isinstance(value, dict):
                flattened_dict.update(flatten_json(value, new_key, sep))

            # If value is a list, iterate through the list and flatten the dictionaries inside
            elif isinstance(value, list):
                for i, item in enumerate(value):
                    # Recursively flatten if the item is a dictionary
                    if isinstance(item, dict):
                        flattened_dict.update(flatten_json(item, f"{new_key}{sep}{i}", sep))
                    else:
                        flattened_dict[f"{new_key}{sep}{i}"] = item

            # If it's neither a dict nor a list, just add the value
            else:
                flattened_dict[new_key] = value

    return flattened_dict

In [102]:
data

{'isHardLoggedIn': False,
 'productDetails': {'productId': 'p810736139',
  'skuId': '810736139',
  'productType': 'svcProduct',
  'isOnDisplay': False,
  'sellType': '',
  'promotionalMessage': '',
  'displayName': 'Arlis Putty Sofa',
  'brandDisplayName': '',
  'displayShowSkuShelfValue': '',
  'productSize': '&amp;nbsp;',
  'curFavStore': '',
  'isSkuBopisEligible': True,
  'powerReviewsEnabled': True,
  'isComingSoon': False,
  'displayLeaseInfo': True,
  'writeAReviewLabel': 'Write a Review',
  'askTheFirstQuestionLabel': 'Ask the First Question',
  'displayQAndA': False,
  'financingTermContent': '&lt;div class=&#034;blcc-learn-more-pdp blcc-wrapper&#034; data-months=6 data-category=&#034;product-cost&#034; style=&#034;width:100%&#034;&gt;\n    &lt;img src=&#034;/resources/images/blcc.svg&#034; alt=&#034;Big Lots Credit Cards icon&#034; class=&#034;blcc-learn-more-pdp-img&#034; style=&#034;flex:0 0 14%;&#034;/&gt;\n    &lt;div class=&#034;blcc-learn-more-pdp-content&#034;&gt;\n   

In [107]:
paths = {
    'name': "$..component",
    'url': "$..baseUrl",
    'id': "$..productId",
    'currency': "$..currency",
    'price_info_final_price': "$..priceConfig.priceFormat.pattern",  # Example for price
    'country_code': "$..country_code",
    'type': "$..productType",
    'images': "$..item.image",
    'productcode': "$..productId"
}

In [105]:
extracted_data = {}
for key, path in paths.items():
    jsonpath_expr = parse(path)
    matches = jsonpath_expr.find(data)
    extracted_data[key] = [match.value for match in matches]

In [106]:
print(json.dumps(flattened_output, indent=4))

{
    "skuId": "810736139",
    "displayName": "Arlis Putty Sofa",
    "brandDisplayName": "",
    "productSize": "&amp;nbsp;",
    "shippingRateLabel": "Same-Day Delivery",
    "productImageList_0_imageSrcSmall": "https://assets.biglots.com/is/image/biglots/810736139-A0-3",
    "productImageList_0_imageSrcMedium": "https://assets.biglots.com/is/image/biglots/810736139-A0-3",
    "productImageList_0_imageSrcLarge": "https://assets.biglots.com/is/image/biglots/810736139-A0-3",
    "productImageList_0_imgForStructuredData": "https://assets.biglots.com/is/image/biglots/810736139-A0-3",
    "productImageList_0_altText": "Arlis Putty Sofa",
    "productImageList_0_isVideo": false,
    "productImageList_0_youtubeUrl": "",
    "productImageList_1_imageSrcSmall": "https://assets.biglots.com/is/image/biglots/810736138_810736139-A0-2",
    "productImageList_1_imageSrcMedium": "https://assets.biglots.com/is/image/biglots/810736138_810736139-A0-2",
    "productImageList_1_imageSrcLarge": "https://